<a href="https://colab.research.google.com/github/Fabricio1618033/Vis-o-Computacional/blob/main/POLIFEMO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#encoding: utf-8
#C:\Python37\python.exe polifemo.py
import cv2, re, sys, time, threading
import numpy as np
from datetime import datetime, date
import pandas as pd
import matplotlib.pyplot as plt
#===# ART #===#
print('  ________   ________   __          _____')
print(' (   ___  ) (   ___  ) (  )        (_   _)')
print(' |  (   ) | |  (   ) | |  |          | |')
print(' |  (___) | |  |   | | |  |          | |                                 _____')
print(' |  ______) |  |   | | |  |          | |       ___    ____   __ __ __   (  _  )')
print(' | (        |  (___) | |  (_____     | |      (  _)  / __ \ (  |_ |  \  | (_) |')
print(' | )        |        | |        \   _| |_   __| |_  (  ___/ |  | | |  | |     |    @@')
print(' |/         (________) (_________) (_____) (__  __)  \____) (__) (_)__) (_____)    @@')
print('                                              | |	  @@@@    @@@@   @@@@@   @@@@@   @   @@  @ @@@    @@@@')
print('                                              | |	 @@   @       @  @@   @  @@@@@   @   @@  @@   @       @')
print('                                             (__)	 @@       @@@@@  @@@@@    @@     @   @@  @        @@@@@')
print('                                                 	 @@   @  @    @  @@       @@  @  @   @@  @       @    @')
print('                                                 	  @@@@    @@@@@  @@         @@    @@@@@  @        @@@@@')
print('###############################################################################################################')


def principal ():
	#===#VARIAVEIS GLOBAIS#===#
	#---#divida a matriz#---#
	print(' Divida a tela em linhas e colunas para marcação dos sensores. Se a divisão não for exata, apresentará um erro.')
	
	var_div_linhas = input(r' Entre com o n° de linhas: ') 
	var_div_colunas = input(r' Entre com o n° de colunas: ') 
	n_sensores = int(input(r'Ativar quantos sensores? - '))
	
	#---#delimitando area de detecção#---#
	def gera_sensor(n):
		tipos = [r' Entre com o nome da direção : ',
			r' Entre com a coordenada x do 1° sensor: ',
			r' Entre com a coordenada y do 1° sensor: ',
			r' Qual a sensibilidade do sensor? (recomendo 0.7%): ']
		f = 0
		lista = []
		while f < n:
		
			lista.append('@'+(input(tipos[0]+'sensor '+str(f+1)+': '))+'@')
			lista.append(input(tipos[1]+'sensor '+str(f+1)+': '))
			lista.append(input(tipos[2]+'sensor '+str(f+1)+': '))
			lista.append(input(tipos[3]+'sensor '+str(f+1)+': '))
			print(' ')
		
			f += 1
		return(lista)
	
	variaveis = gera_sensor(n_sensores)
	
	'''
	#---#delimitando area de detecção#---#
	str_sentido = input(r' Entre com o nome da direção 1: ')
	trix_a = input(r' Entre com a coordenada x do 1° sensor: ') #2
	triy_a = input(r' Entre com a coordenada y do 1° sensor: ') #8
	
	print('---')
	#---#
	sensibilidade = input(r' Qual a sensibilidade do sensor? (recomendo 0.7%): ') # valor recomendado de 0.7%
	#---#
	'''
	def select_cam(a):
		if a == a:
			return(a)
		elif a == 'ip':
			return('http://admin:@192.168.0.13/media/?action=stream.asp')
		else:
			print('Entre com 1 ou 2 ou 3')
	print('Selecione a camera:')
	print('0 - webcam nativa')
	print('qualquer numero inteiro - webcam adcional')
	print('ip - camera IP')
	str_camera = input('Entre com os n° acima: ')
	var_camera = (select_cam(int(str_camera)))
	print('---')
	#---#
	str_salva = input('Ativa a função de salvar: sim ou não? ')
	var_salva =(str_salva)
	print('---')
	#---#
	print('Exibir grade?')
	var_grade = input('Entre com on ou off: ')
	print('---')
	#---#
	str_inv_tela = input('inverter a tela: sim ou não? ')
	var_invert_tela = (str_inv_tela)
	print('---')




	#===#FUNÇÕES DIVERSAS#===#
	def salva_csv(df):    # Salva o dataframe gerado.
		data_geral = time.strftime('%d-%a-%Y-%H_%M_%S')
		df.to_csv('C:\\Users\\User\\OneDrive\\PROGRAMAS\\Captura de movimento\\reg_mov\\dados\\teste_ARAUJO_'+data_geral+'.csv')
		print ('Arquivo salvo em C:\\Users\\User\\OneDrive\\PROGRAMAS\\Captura de movimento\\dados\\teste_ARAUJO_'+data_geral+'.csv')
	
	def salva_teste_csv(df,str_texto):    # Salva o dataframe gerado.
		df.to_csv('C:\\Users\\User\\OneDrive\\PROGRAMAS\\Captura de movimento\\reg_mov\\dados\\'+str_texto+'.csv')
		print ('Arquivo salvo em C:\\Users\\User\\OneDrive\\PROGRAMAS\\Captura de movimento\\dados\\'+str_texto+'.csv')


	def calculaDiferenca(img1,img2,img3):    #Calcula a diferença entre os frames do video e retorna a imagem preta e branco quando há movimento.  
		d1 = cv2.absdiff(img3,img2)          #ESTUDAR ESSA FUNÇÃO MAIS PROFUNDAMENTE.
		d2 = cv2.absdiff(img2,img1)
		imagem = cv2.bitwise_and(d1,d2)
		s,imagem = cv2.threshold(imagem,35,255,cv2.THRESH_BINARY)
		return (imagem)

	def sensor(matrix_cam, triy_a, trix_a, sensibilidade,str_sentido):
		ajuste = sensibilidade
		#pega um array total(tela returnada pela camera e seleciona um pedaço dela.)
		def xadrez (matrix,select_y,select_x):                  
			def fatia_horizontal (matrix,dividir,cell_y):       #Ao dividir a matriz tenho que garantir que a divisão tenha resto zero
				sub_matrix_h = np.split(matrix,dividir)[cell_y-1]     #senão, não forma sub-matrizes iguais.
				return (sub_matrix_h)
			def fatia_vertical (matrix,dividir,cell_x):
				sub_matrix_v = np.hsplit(matrix,dividir)[cell_x-1]
				return (sub_matrix_v)
			matriz_select = fatia_vertical(fatia_horizontal(matrix,int(var_div_linhas),select_x),int(var_div_colunas),select_y) 
			return (matriz_select)
		#===# Laços de captura de movimento ***CORAÇÃO*** #===#
		# reduz_ruido => por meio do metodo 'shape' da matriz em questão obtenho o numero total de pixel da tela, 
		# neste cado da webcam do notebook é y = 480 x = 640 dando um total de 307200 pixes. Porem para controlar 
		# a sensibilidade da detecção do registro de movimento que é feito com a soma das matrizes alteradas por 
		# meio da função 'mov' temos que multiplicar os pixes pelo valor de branco, que é de 255, e condicionar o 
		# registro a capacidade de preenchimanto de pixes branco na tela, pois senão ao menor sinal de movimento, 
		# digo na presença de um unico pixel branco haveria o registro.
		# Na função 'reg_mov' o metodo 'append' alimenta listas pré-posicionadas fora do loço while que seram usadas 
		# para formação do dataframe.
		# As listas são compostas pela engenharia dos dados que queremos obter seja o tempo como data e hora, o numero 
		# de pixes ativados e o nome da area monitorada.	
		def reg_mov(matrix,str_id):             # id = string para identificar o setor monitorado pela camera.
			# A função 'mov' soma os numeros da matriz afim de diferenciar uma matriz da outra.
			# Uma matriz sem movimento, que devido a função 'calculaDiferenca' fica toda escura logo toda formada de zeros terá soma zero. E uma matriz cuja  
			# uma parte dela houve movimento, apresenta pixes brancos com valor de 255 cada, logo na soma dos pixes teremos valores distintos para cada frame.
			# lista sem movimento [0,0,0,0,0] = 0   lista com movimento [0,0,255,255,0,0] = 510
			# Obs: Para obter o numero de pixel com o valor da soma das listas é só obter a divisão por 255 ex: 510/255 = 2 pixel
			def mov (array):
				lista = array.flat[:]	#transformou em uma matriz unidirecional"como uma lista".
				soma = np.sum(lista)
				return (soma)
			def reduz_ruido(porcento):
				limite_preenchimento = (((((matrix.shape[0])*(matrix.shape[1]))*255.0)*porcento)/100.0)
				return (limite_preenchimento)
			if mov(matrix) >= reduz_ruido(float(ajuste)):                                  # Registro por pixes brancos 
				print (str(str_id),'-',mov(matrix),'pixes','-','tempo','-',dia)    # Feito para monitorar a execução do sistema ativo.
				acao.append(str(str_id))
				pixes.append('!'+str(mov(matrix))+'!') # O caracter '!' está sendo usando como marcação para ajuste dos dados na função limpa_dataframe.
				tempo.append('?'+str(time.time())+'?') # O caracter '?' está sendo usando como marcação para ajuste dos dados na função limpa_dataframe.
				data.append(str(datetime.now())) # O caracter '&' está sendo usando como marcação para ajuste dos dados na função limpa_dataframe.	
			
		reg_mov(xadrez(matrix_cam,int(triy_a),int(trix_a)),str_sentido)

	# OBS: Os dados são registrados independente de mostrar a imagem bloqueando o "imshow". Ou não colocando a função 'tela' no loop while.
	# O movimento ainda é registrado!
	def tela(legenda,matrix,tela_y,tela_x):
		
		#y,x = (matrix.shape)
		shape_y = tela_y #720
		shape_x = tela_x #1280
		linhas = int(var_div_linhas)
		colunas = int(var_div_colunas)
	
		def grade(y,x,colunas,linhas):
			for i in range(1,colunas):
				cv2.line(matrix, (int(x/colunas)*i,0),(int(x/colunas)*i,y), (0, 0, 255),2)
			for i in range(1,linhas):
				cv2.line(matrix, (0,int(y/linhas)*i),(x,int(y/linhas)*i), (0, 0, 255),2)
	
		if var_grade == 'on':
			grade(shape_y,shape_x,colunas,linhas)   #ipcam
			#grade(480,640,40,4)     #webcam
		elif var_grade == 'off':
			pass
	
		#MARCA DE MOVIMENTO#
		
		def retangulo_red(coluna,linha,largura,altura):
			return cv2.rectangle(matrix, ((coluna*largura),(linha*altura)-altura),((coluna*largura)-largura,(linha*altura)), (0, 0, 255),2)
		def retangulo_verde(coluna,linha,largura,altura):
			return cv2.rectangle(matrix, ((coluna*largura),(linha*altura)-altura),((coluna*largura)-largura,(linha*altura)), (0, 255, 0),2) 
		def retangulo_amarelo(coluna,linha,largura,altura):
			return cv2.rectangle(matrix, ((coluna*largura),(linha*altura)-altura),((coluna*largura)-largura,(linha*altura)), (0, 255, 255),2)  
	
		#---ponto ancora---#
		retangulo_amarelo(1,1,int(shape_x/colunas),int(shape_y/linhas)) # marca de grade
	
		#---#marca dos sensores#---#
		for i in range(n_sensores):
			if i%2 == 0:
				retangulo_amarelo(int(variaveis[(i*4)+1]),int(variaveis[(i*4)+2]),int(shape_x/colunas),int(shape_y/linhas))
			else:
				retangulo_verde(int(variaveis[(i*4)+1]),int(variaveis[(i*4)+2]),int(shape_x/colunas),int(shape_y/linhas))
		if var_invert_tela == 'não':	
			cv2.imshow(legenda,matrix)                  # indicado para ipcam.
		elif var_invert_tela == 'sim':
			cv2.imshow(legenda,(cv2.flip(matrix,1)))	# espelha a imagem da camera.

	
	def limpa_dataframe(df):
		clear = re.compile(r'\"|\(|\[|\'|\>|\<|\]|\;|\)|\!|\?|\@') # Retirar os marcadores e elementos desnecessarios no dataframe.
		#--- Setor analizado
		dados_setor = []
		dados_data = []
		dados_pixes = []
		dados_tempo = []
	
		busca_setor = re.compile (r'\@.*\@')
		busca_data = re.compile(r'\d\d\d\d\-\d\d\-\d\d \d\d\:\d\d\:\d\d\.\d+')
		busca_pixes = re.compile(r'\!.*\!')
		busca_tempo = re.compile(r'\?.*\?')
		for i in df:
			minerio = busca_setor.findall(str(i))
			dados_setor.append(clear.sub('',(str(minerio))))
			minerio2 = busca_data.findall(str(i))
			dados_data.append(clear.sub('',(str(minerio2))))
			minerio3 = busca_pixes.findall(str(i))
			dados_pixes.append(clear.sub('',(str(minerio3))))
			minerio4 = busca_tempo.findall(str(i))
			dados_tempo.append(clear.sub('',(str(minerio4))))
		df_setor = pd.Series(dados_setor, index=None)
		df_pixes = pd.Series(dados_pixes, index=None)
		df_tempo = pd.Series(dados_tempo, index=None)
		df_data = pd.Series(dados_data, index=None)
	
		dataframe_limpo = pd.concat([df_setor,df_pixes,df_tempo,df_data], axis=1)
		dataframe_limpo.columns = ['setor','pixes','tempo','data']
	
		return (dataframe_limpo)	

	#===#backup dos dados#===#	
	def salva_intervalos(a):
		#f = 0
		while True:#f < a: #datetime.now() < datetime(2019, 3, 30, 11, 35, 0):
			time.sleep(1200)
			dados = list(zip(acao,pixes,tempo,data))
			df_bruto = pd.Series(dados)
			salva_teste_csv(limpa_dataframe(df_bruto),'backup')
			#f = f + 1
			
	thread_salva_inter = threading.Thread(target=salva_intervalos, args=[36])


	#===#ACESSO A CAMERA E REGISTRO DE MOVIMENTO#===#
	#===#ACESSO A CAMERA E REGISTRO DE MOVIMENTO#===#
	#===#ACESSO A CAMERA E REGISTRO DE MOVIMENTO#===#		

	#webcam = cv2.VideoCapture('http://admin:@192.168.137.133/media/?action=stream.asp')  # Para conexão com o wifi hotspot.
	#webcam = cv2.VideoCapture('http://admin:@192.168.0.13/media/?action=stream.asp')     # Para conexã com wifi de casa.
	webcam = cv2.VideoCapture(var_camera)  # instancia o uso da webcam... em caso de mais cameras selecionar com numeros diferentes como 0...1...2...3...etc
								# ou camera ip com a url stream como 'http://admin:@192.168.0.13/media/?action=stream.asp'
	cv2.namedWindow("Tela de captura", cv2.WINDOW_NORMAL)  # cria uma janela redimensionavel (cv2.WINDOW_NORMAL)

	# Faz a leitura inicial de imagens para uso na função 'calculaDiferenca'
	ultima        = cv2.cvtColor(webcam.read()[1],cv2.COLOR_RGB2GRAY)
	penultima     = ultima
	antepenultema = ultima

	check, frames = webcam.read()


	# Listas para transporte dos dados coletados
	
	acao =  []#['start']
	pixes = []#['!'+str(0.0)+'!']
	tempo = []#['?'+str(time.time())+'?']
	data =  []#['&'+str(datetime.now())+'&']


	#===#INICIO DA THREAD SECUNDARIA#===#
	thread_salva_inter.start()


	print ('Para SAIR... click na tela de video e digite "S".')

	while True:
		# Variaveis diversas.
		check, frame = webcam.read()     # tamanho da tela y = 480 x = 640
	
		antepenultema = penultima   # com essa linha bloqueada a sensibilidade fica melhor.
		penultima     = ultima
		ultima        = cv2.cvtColor(webcam.read()[1],cv2.COLOR_RGB2GRAY)
	
		matriz = calculaDiferenca(antepenultema,penultima,ultima)  # ORIGEM DE TODA MANIPULAÇÃO DAS IMAGENS.
		
		hoje = datetime.now()  
		dia = time.strftime('%Y-%m-%d %H:%M:%S')
		y,x = matriz.shape
	
	
		# EXIBIÇÃO EM TEMPO REAL...
		# O uso da variavel 'matriz' na função 'tela' a exibição é preta com movimento branco. 
		# E a variavel 'frame' do metodo 'webcam.read()' exibi o video em cores.
		tela('Tela de captura',matriz,y,x)         # Preto com movimento em branco. As marcações não aparecem aqui pois a função calculaDiferenca a escurece.
		tela('Tela de captura',frame,y,x)          # Video em cores. obs:Marcações colocadas na função 'tela'.
	
	
	
	
		#---EXECUÇÃO
		# xadrex(matriz usada, escolhe a coluna, escolhe a linha que será visualizada e detectada)
	
	
		# FORMAÇÃO DOS SENSORES
		for i in range(n_sensores):
			sensor(matriz, int(variaveis[(i*4)+1]), int(variaveis[(i*4)+2]), float(variaveis[(i*4)+3]),variaveis[(i*4)])
	
	
		if cv2.waitKey(1) == ord('s'):   # APERTE 'S' PARA SAIR!!!
	
			print ('')
			#print (hoje,'atividade da tela')
		
			break
	cv2.destroyWindow("Tela de captura")

	# Ao contrário do Python 2, a função zip no Python 3 retorna um iterador . Iteradores só podem ser esgotados (por algo como fazer uma lista deles) uma vez. 
	# O propósito disso é economizar memória gerando apenas os elementos do iterador conforme necessário, em vez de colocar tudo na memória de uma só vez. 
	# Se você quiser reutilizar o seu objeto zipado, basta criar uma lista dele. 

	dados = list(zip(acao,pixes,tempo,data))  # no python2 funciona normal...mesmo sem o 'list' na frente do zip. No python 3 só funciona com o 'list' na frente.
	df_bruto = pd.Series(dados)




	print(limpa_dataframe(df_bruto))
	if var_salva == 'sim':
		salva_csv(limpa_dataframe(df_bruto))
	elif var_salva == 'não':
		pass
	sys.exit()

#===# EXECUÇÃO #===#

principal()